In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import os

from forecasting_utils import calculate_metrics, DataManagerSimplified, LGBMForecaster

import warnings
warnings.filterwarnings("ignore")

## ЗАДАНИЕ: Прогнозирование продаж в магазинах офлаин-ритейлера в США 

В качестве экзаменационного задания вам предстоит построить модель прогнозирования спроса(продаж) на товары в магазинах офлаин-ритейлера в США . Всего в датасете 3 магазина, в каждом магазине по 15 артикулов (товаров). Вам нужно выбрать 1 магазин (любой из 3).

Прогнозировать продажи нужно на неделю, на месяц и на квартал.

В качестве дополнительной информации вам переданы данные о цене товара (меняются раз в неделю, а также о праздниках в США).

Задание 1. Реализовать класс, который умеет:

1) предобрабатывать исходные данные в удобный формат;

2) обучаться для задачи прогнозирования;

3) оценивать качество своих прогнозов;

4) сохранять модели и подгружать их;

5) Прогнозировать продажи на неделю, на месяц и на квартал.

Должна быть рабочая программа, которая делает инференс (прогнозирование на произвольном тестовом датасете (аналогично тому, который есть у вас)).

Задача 2. Подготовить отчёт о решении данной задачи в виде jupyter ноутбука. В отчёте, в частности, вы должны ответить на следующие вопросы:

1) Какие методы предобработки данных вы использовали?

2) Какие модели пробовали? Почему пробовали именно их?

3) Как вы проверяете качество модели? На каких данных? Какие метрики используюте? Чем обусловлен выбор именно этих метрик?

4) Какое итоговое качество модели на тестовом датасете?

## Задание 1.

Классы определены в forecasting_utils.py

Проверим работу на тестовых данных

In [12]:
# --- Параметры ---
STORE_ID = 'STORE_1'
MODELS_DIR = f"models_{STORE_ID}_lgbm_simple_v3" # Новая папка

# Пути к ОБУЧАЮЩИМ файлам
SALES_TRAIN_PATH = 'data/shop_sales.csv'
CALENDAR_TRAIN_PATH = 'data/shop_sales_dates.csv'
PRICES_TRAIN_PATH = 'data/shop_sales_prices.csv'

# Пути к ОЦЕНОЧНЫМ (EVALUATION) файлам
SALES_EVAL_PATH = 'test_data/shop_sales_test.csv'
CALENDAR_EVAL_PATH = 'test_data/shop_sales_dates_test.csv'
PRICES_EVAL_PATH = 'test_data/shop_sales_prices_test.csv'

# --- Создаем Dummy оценочные файлы, если их нет ---
eval_data_dir = 'test_data'; os.makedirs(eval_data_dir, exist_ok=True)
def create_dummy_if_missing(src_path, dest_path):
    if not os.path.exists(dest_path):
        try:
            if os.path.exists(src_path): pd.read_csv(src_path).head(0).to_csv(dest_path, index=False); print(f"  Создан dummy файл: {dest_path}")
            else: print(f"  Ошибка: Исходный файл {src_path} не найден.")
        except Exception as e: print(f"  Ошибка при создании dummy файла {dest_path}: {e}")
create_dummy_if_missing(SALES_TRAIN_PATH, SALES_EVAL_PATH)
create_dummy_if_missing(CALENDAR_TRAIN_PATH, CALENDAR_EVAL_PATH)
create_dummy_if_missing(PRICES_TRAIN_PATH, PRICES_EVAL_PATH)

# Параметры для финальной модели LGBM
final_lgbm_params = {
    'objective': 'regression_l1', 'n_estimators': 57, 'learning_rate': 0.1,
    'num_leaves': 15, 'feature_fraction': 0.8, 'bagging_fraction': 0.9,
    'bagging_freq': 1, 'verbose': -1, 'n_jobs': -1, 'seed': 42,
    'boosting_type': 'gbdt'
}


In [13]:
# --- Этап 1: Обучение или Загрузка Модели ---
print(f"\n--- Этап 1: Обучение или Загрузка Модели для {STORE_ID} ---")
forecaster = LGBMForecaster(store_id=STORE_ID, model_dir=MODELS_DIR)

models_loaded = forecaster.load_model()
if not models_loaded:
    print("\n--- Модели не загружены, начинаем обучение ---")
    data_manager = DataManagerSimplified(store_id=STORE_ID)
    train_base_df = data_manager.get_base_data(
        sales_path=SALES_TRAIN_PATH, calendar_path=CALENDAR_TRAIN_PATH, prices_path=PRICES_TRAIN_PATH
    )
    if train_base_df is not None:
        forecaster.fit(train_base_df=train_base_df, lgbm_params=final_lgbm_params)
    else: print("Обучение не выполнено из-за ошибки подготовки данных.")
else:
    print("\n--- Компоненты модели успешно загружены ---")
    if forecaster.history_df is not None: print(f"  История загружена из файла: {forecaster.history_df.shape}")
    else: print("  Внимание: История не была загружена."); forecaster.is_fitted = False # Нельзя предсказывать без истории


--- Этап 1: Обучение или Загрузка Модели для STORE_1 ---
LGBMForecaster инициализирован для магазина: STORE_1. Папка модели: 'models_STORE_1_lgbm_simple_v3'
  Загрузка компонентов модели из models_STORE_1_lgbm_simple_v3...
    model_lgbm загружен.
    features загружен.
    categorical_features загружен.
    history_df загружен.
  Компоненты модели успешно загружены.

--- Компоненты модели успешно загружены ---
  История загружена из файла: (27285, 4)


In [11]:
# --- Этап 2: Инференс и Оценка ---
if forecaster.is_fitted:
    print(f"\n--- Этап 2: Инференс и Оценка для {STORE_ID} ---")
    data_manager_eval = DataManagerSimplified(store_id=STORE_ID)

    print("\nПодготовка базовых данных для периода оценки...")
    eval_base_df = data_manager_eval.get_base_data(
        sales_path=SALES_EVAL_PATH, calendar_path=CALENDAR_EVAL_PATH, prices_path=PRICES_EVAL_PATH
    )

    if eval_base_df is not None:
        print(f"  Базовые данные для оценки подготовлены: {eval_base_df.shape}")

        # 2. Генерация прогнозов, передавая ГОТОВЫЙ базовый DataFrame
        print("\nГенерация прогнозов...")
        predictions_df = forecaster.predict(predict_base_df=eval_base_df) # <<< ИЗМЕНЕННЫЙ ВЫЗОВ

        # 3. Оценка прогнозов
        if predictions_df is not None:
            # Для оценки нужны фактические продажи 'cnt', которые есть в eval_base_df
            evaluation_results = forecaster.evaluate(
                y_true_df=eval_base_df[['date', 'item_id', 'cnt']],
                y_pred_df=predictions_df
            )
            if evaluation_results:
                print("\n--- Результаты оценки модели на оценочном наборе ---")
                eval_df = pd.DataFrame(evaluation_results)
                print(eval_df.round(4))
        else:
            print("  Не удалось сгенерировать прогнозы.")
    else:
        print("  Ошибка: Не удалось подготовить базовые данные для оценки.")
else:
    print("Модель не обучена или не загружена / отсутствует история. Тестирование невозможно.")

print("\n--- Скрипт завершен ---")


--- Этап 2: Инференс и Оценка для STORE_1 ---
DataManagerSimplified инициализирован для магазина: STORE_1

Подготовка базовых данных для периода оценки...

--- Получение базовых данных для sales: shop_sales_test.csv ---
  Загрузка календаря из: test_data/shop_sales_dates_test.csv
  Загрузка цен из: test_data/shop_sales_prices_test.csv
    Календарь загружен: (150, 14)
    Цены загружены и отфильтрованы: (345, 3)
  Предобработка продаж из: shop_sales_test.csv...
    Продажи предобработаны: (1410, 3)
  Объединение данных...
  Базовый DataFrame готов: (1410, 7)
  Базовые данные для оценки подготовлены: (1410, 7)

Генерация прогнозов...

--- Генерация прогнозов LGBM ---
  Создание признаков для прогноза...
  Создание признаков LGBM. Режим обучения: False. Входная форма: (1410, 7)
    Расчет лагов/окон с использованием history_df (режим прогноза)...
    Размер объединенного df для расчета: (28695, 3)
    Заполнение NaN и оптимизация типов для 21 признаков...
  Создание признаков LGBM завер

## Задание 2.

В процессе решения были выполнены этапы анализа данных, предобработки, выбора и обучения моделей, оценки качества и реализации итогового решения в виде классов.

### 1) Какие методы предобработки данных вы использовали?



В ходе работы были применены следующие основные методы предобработки данных:

1.  **Загрузка и объединение данных:**
    *   Загружены три исходных файла: `shop_sales.csv` (продажи), `shop_sales_dates.csv` (календарь), `shop_sales_prices.csv` (цены).
    *   Данные отфильтрованы для целевого магазина `STORE_1`.
    *   Данные объединены в единый DataFrame (`base_df` внутри классов) с использованием `date_id` (для добавления даты к продажам) и `wm_yr_wk` (для добавления цен).

2.  **Обработка пропущенных значений:**
    *   **Цены (`sell_price`):** Пропуски, возникающие из-за того, что цены меняются раз в неделю, а продажи ежедневные, заполнялись методом `ffill().bfill()` (сначала прямое, затем обратное заполнение) **внутри группы каждого товара (`item_id`)**. Оставшиеся редкие пропуски (если товар никогда не имел цены) заполнялись нулем. При генерации признаков для прогноза недостающие будущие цены также заполнялись последним известным значением из истории или нулем.
    *   **События (`event_name_1`, `event_type_1`):** Пропуски (означающие отсутствие события) заполнялись строковыми значениями 'NoEvent' и 'NoType' соответственно. Столбцы `event_name_2` и `event_type_2` были полностью удалены на этапе EDA из-за крайне малого количества не-пропущенных значений (менее 0.3%).

3.  **Создание признаков (Feature Engineering) для модели LightGBM:**
    *   **Календарные признаки:** Из столбца `date` извлечены: день недели (`dayofweek`, `wday`), день месяца (`dayofmonth`), неделя года (`weekofyear`), день года (`dayofyear`), год (`year`). Признак `month` был исключен на этапе оптимизации как неважный.
    *   **Лаговые признаки:** Созданы признаки со значениями продаж (`cnt`) за прошлые периоды (`cnt_lag_N`, где N = 7, 14, 21, 28, 35, 90 дней). Расчет производился отдельно для каждого товара.
    *   **Признаки скользящих окон:** Рассчитаны скользящее среднее (`cnt_roll_mean_W`) и стандартное отклонение (`cnt_roll_std_W`) продаж за окна W = 7, 14, 28 дней. При расчете использовался сдвиг на 7 дней (`base_shift=7`) для предотвращения утечки данных при обучении.
    *   **Кодирование ID товара (`item_id`):** Вместо `LabelEncoder` использовался упрощенный подход: из строкового ID извлекались последние 3 цифры, преобразовывались в целое число и сохранялись в признаке `item_id_numeric`. Этот признак передавался в LightGBM как **категориальный**, используя его числовые значения (0-14 для STORE_1).
    *   **Признаки событий:** На этапе оптимизации признаки событий (`event_name_*`, `event_type_*`) были **удалены** из финального набора, так как анализ важности показал, что модель эффективно использует календарные признаки (`dayofyear`, `weekofyear` и т.д.) для учета влияния регулярных праздников.
    *   **Другие признаки:** Признак `cashback` был также удален как неважный. Признак `sell_price` был оставлен.

4.  **Обработка NaN после Feature Engineering:** Строки с NaN, появившиеся в **обучающей** выборке из-за расчета лагов и скользящих окон, были **удалены** (`dropna()`) перед подачей в модель. При генерации признаков для **прогноза** пропуски в лагах/окнах заполнялись нулями.


### 2) Какие модели пробовали? Почему пробовали именно их?

Были опробованы и сравнены модели из разных классов:

1.  **Baseline (Сезонный наивный):** Простейшая модель, предсказывающая продажи текущего дня как продажи того же дня недели неделю назад (`cnt[D] = cnt[D-7]`).
    *   *Причина выбора:* Необходим для оценки прироста качества от более сложных моделей; хорошо подходит для данных с выраженной недельной сезонностью.
2.  **Статистические модели временных рядов:**
    *   **ETS (Error, Trend, Seasonality):** Экспоненциальное сглаживание. Моделирует ряд как комбинацию уровня, тренда (аддитивного/мультипликативного, затухающего/незатухающего) и сезонности (аддитивной/мультипликативной).
    *   **AutoARIMA:** Автоматически подбирает порядки (p,d,q)(P,D,Q) для модели ARIMA/SARIMA, учитывая стационарность и сезонность (задавали `sp=7`).
    *   **TBATS:** Более сложная модель на основе ETS, способная обрабатывать несколько типов сезонности (мы задавали недельную и месячную), преобразование Бокса-Кокса, ARMA-ошибки.
    *   *Причина выбора:* Это классические и хорошо зарекомендовавшие себя подходы для одномерных временных рядов, способные улавливать основные компоненты ряда (тренд, сезонность, автокорреляцию).
3.  **Модели с регрессорами:**
    *   **Prophet:** Разработана Facebook, хорошо обрабатывает множественную сезонность, праздники (автоматически или заданные пользователем), пропуски и выбросы. Позволяет включать дополнительные регрессоры.
    *   **Orbit (DLT):** Байесовская модель (Damped Local Trend), позволяющая включать регрессоры и получать оценку неопределенности прогноза.
    *   *Причина выбора:* Позволяют явно учесть влияние внешних факторов (цены, праздники, кэшбэк) и часто дают хорошие результаты "из коробки".
4.  **Модели машинного обучения:**
    *   **LightGBM:** Реализация градиентного бустинга. Очень быстрая и эффективная, хорошо работает с табличными данными, способна улавливать сложные нелинейные зависимости и взаимодействия между признаками.
    *   *Причина выбора:* Возможность использования большого количества созданных признаков (календарных, лаговых, оконных, цены, ID товара); потенциал для высокой точности; скорость. Была выбрана как основной кандидат после анализа EDA и важности признаков. Проводилась оптимизация признаков и подбор гиперпараметров.


### 3) Как вы проверяете качество модели? На каких данных? Какие метрики используюте? Чем обусловлен выбор именно этих метрик?


1.  **Данные для проверки:** Качество моделей проверялось на **отложенном тестовом (оценочном) наборе данных**. Этот набор был создан путем отделения **последних 90 дней** из всего доступного временного диапазона данных. Обучение проводилось на всех данных до начала этого 90-дневного периода. Это гарантирует, что модель оценивается на данных, которые она не видела во время обучения, и имитирует реальную ситуацию прогнозирования будущего.
2.  **Горизонты прогнозирования:** Оценка проводилась для трех требуемых горизонтов:
    *   **Неделя:** Первые 7 дней тестового набора.
    *   **Месяц:** Первые 30 дней тестового набора.
    *   **Квартал:** Весь тестовый набор (первые 90 дней, но в логах финального запуска класса указано 94 дня, что требует уточнения - возможно, тестовый набор был чуть больше или расчет сдвинулся).
3.  **Метрики:** Использовался набор метрик для всесторонней оценки:
    *   **MAE (Mean Absolute Error):** Основная метрика для оценки средней абсолютной ошибки в единицах проданного товара. Легко интерпретируется и менее чувствительна к выбросам, чем RMSE. Выбрана как ключевая для сравнения моделей.
    *   **RMSE (Root Mean Squared Error):** Дополнительная метрика абсолютной ошибки, также в единицах товара. Сильнее штрафует за большие ошибки, полезна для понимания их масштаба.
    *   **sMAPE (Symmetric Mean Absolute Percentage Error):** Метрика относительной ошибки (в процентах, шкала 0-200%). Выбрана вместо MAPE из-за наличия нулевых продаж в данных, так как sMAPE корректно обрабатывает нули в знаменателе. Позволяет сравнивать точность на товарах с разным уровнем продаж.
    *   **R² (Coefficient of Determination):** Используется для контекста, показывая долю дисперсии, объясненную моделью, по сравнению с наивным прогнозом среднего значения. Отрицательные значения указывают, что модель хуже прогноза среднего. Полезна для понимания общей адекватности модели, особенно на длинных горизонтах.
    *   *Обоснование выбора:* Комбинация MAE/RMSE дает представление об абсолютной точности, sMAPE - об относительной (особенно важно при разных масштабах продаж), а R² - об общей объясняющей способности модели.



### 4) Какое итоговое качество модели на тестовом датасете?

Финальной моделью была выбрана **LightGBM с оптимизированными признаками и гиперпараметрами** (`model_lgb_final_simplified` в коде экспериментов, реализованная в классе `LGBMForecaster`).

Ее качество на **оценочном наборе данных** (последние ~90-94 дня), полученное в **последнем запуске скрипта с классом**, составило:

| Горизонт | MAE     | RMSE    | sMAPE    | R2      |
| :------- | :------ | :------ | :------- | :------ |
| Неделя   | 3.7358  | 4.7791  | 72.3926  | -0.1135 |
| Месяц    | 11.6980 | 14.4285 | 115.3830 | -2.1965 |
| Квартал  | 13.3238 | 15.7016 | 149.1814 | -2.3451 |

**Интерпретация итогового качества:**

*   **Недельный прогноз:** Модель показывает **хорошую точность** по абсолютным метрикам (MAE ~3.7 единицы товара) и приемлемую относительную ошибку (sMAPE ~72%). Однако отрицательный R2 говорит о том, что модель все еще хуже простого среднего при объяснении вариативности на этом горизонте.
*   **Месячный и Квартальный прогнозы:** Качество модели **резко падает** на более длинных горизонтах. Ошибки MAE/RMSE становятся очень большими, sMAPE превышает 100%, а R2 сильно отрицательный. Это указывает на то, что текущая реализация прогнозирования (одношаговая модель с лагами, рассчитанными на основе истории без рекурсии) **не способна** генерировать адекватные прогнозы на срок более одной-двух недель. Результаты экспериментов, показывавшие хорошее качество LGBM на длинных горизонтах, были **ошибочно оптимистичными** из-за утечки данных при генерации признаков для оценки.

**Заключение по качеству:** Реализованный класс `LGBMForecaster` с текущим методом прогнозирования обеспечивает **хорошее качество только для краткосрочных прогнозов (до недели)**. Для прогнозов на месяц и квартал требуется либо принципиально иной подход к моделированию (рекурсивный прогноз, direct-стратегия), либо использование других моделей (например, статистических), которые внутренне поддерживают многошаговый прогноз.
